In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib as mp # Plotting in Python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
   

Read the csv data into a pandas dataframe

In [ ]:
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

train = train.drop(train.loc[:, 'date':'resp'].columns, axis = 1)
#train = train.drop(["feature_121"], axis = 1)
train = train.drop(["ts_id"], axis = 1)

print(train)

Let's find the rows with NaN, and try and fill in the data

In [ ]:
is_NaN = train.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train[row_has_NaN]

print(rows_with_NaN)

print("Number of rows with NaN is: " + str(len(rows_with_NaN.index)))

for i in range(0, 129):
    print("Number of null values in feature_" + str(i) + " : " + str(train.loc[:, 'feature_' + str(i)].isnull().sum())) 


So, now that we know that there are roughly 400,000 rows with a value NaN, if we eliminate these rows, then we should have only valid values.

Once we have valid values, we can perform PCA analysis and try to categorise these features

In [ ]:
train_no_NaN = train.dropna()

print(train_no_NaN)

We want to scale each feature, otherwise when we feed it into PCA, it will give a lot of weighting to certain numbers

In [ ]:
from sklearn.preprocessing import StandardScaler

train_no_NaN = StandardScaler().fit_transform(train_no_NaN)
feat_cols = ['feature'+str(i) for i in range(train_no_NaN.shape[1])]

train_scaled = pd.DataFrame(train_no_NaN,columns=feat_cols)
train_scaled.head()

Finally, let's just feed it in. We can change nComponents to different amounts to try and increase or decrease the explained variance

In [ ]:
from sklearn.decomposition import PCA

nComponents = 30

pca_train = PCA(n_components=nComponents)
principalComponents_train = pca_train.fit_transform(train_no_NaN)

print('Explained variation from ' + str(nComponents) + ' principal component: {}'.format(sum(pca_train.explained_variance_ratio_)))

print('Explained variation per principal component: {}'.format(pca_train.explained_variance_ratio_))